In [133]:
import xgboost as xgb
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# import pickle
from joblib import dump, load

# %matplotlib inline
# import matplotlib.pyplot as plt

In [56]:
# %pwd

In [57]:
train = pd.read_csv("../data/pytrain.csv")
test = pd.read_csv("../data/pytest.csv")

In [58]:
# train = train.replace(to_replace = np.nan, value = -9)
# test = test.replace(to_replace = np.nan, value = -9)
# test.head(20)

In [59]:
train = train.dropna(subset = ['Y'])
train_y = train['Y']
train_id = train['system_key']
dtrain = train.drop(columns = ['Y', 'system_key'])

test = test.dropna(subset = ['Y'])
test_y = test['Y']
test_id = test['system_key']
dtest = test.drop(columns = ['Y', 'system_key', 'Unnamed: 0'])

In [60]:
dtrain.head()

,qtr.num,class.1,class.2,class.3,class.4,tenth_day_credits,scholarship_type,num_courses,pts,attmp,...,tran_major_abbrUWACAD,tran_major_abbrV.A.S,tran_major_abbrWOMEN,major.change,major.change.count,n.unmet,n.unmet.cum,ftFALSE,ftTRUE,ft.creds.over
0,9,0,0,1,0,15.0,0,3,39.0,15.0,...,0,0,0,0,1,0,6,0,1,3.0
1,6,0,1,0,0,15.0,0,3,43.5,15.0,...,0,0,0,0,0,4,9,0,1,3.0
2,2,1,0,0,0,15.0,0,3,45.0,15.0,...,0,0,0,0,0,0,0,0,1,3.0
3,1,1,0,0,0,17.0,0,4,47.5,15.0,...,0,0,0,0,0,0,0,0,1,5.0
4,4,0,1,0,0,17.0,0,4,50.0,15.0,...,0,0,0,0,0,2,6,0,1,5.0


In [61]:
if dtest.shape[1] != dtrain.shape[1]:
    i1 = train.columns
    i2 = test.columns
    [x for x in i2 if x not in i1]
else: 
    print("√ train/test have same number of columns √")

√ train/test have same number of columns √


In [62]:
dtrain = xgb.DMatrix(dtrain, label = train_y, missing = np.NaN)
dtest = xgb.DMatrix(dtest, label = test_y, missing = np.NaN)

/Users/zane/anaconda3/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/zane/anaconda3/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [63]:
param = {'max_depth': 4, 
         'eta': 1, 
         'objective': 'reg:squarederror'}
evallist = [(dtest, 'eval'), (dtrain, 'train')]
nrounds = 3

In [64]:
bst = xgb.train(param, dtrain, nrounds, evallist)

[0]	eval-rmse:0.310271	train-rmse:0.307411
[1]	eval-rmse:0.301429	train-rmse:0.297592
[2]	eval-rmse:0.289436	train-rmse:0.284114


In [67]:
bst.predict(dtest)[:20]

array([3.4236171, 2.9237692, 3.6941247, 3.6941247, 3.4306018, 3.2245584,
       3.0839188, 3.6941247, 2.926634 , 3.3301504, 2.7940795, 2.9122558,
       3.3473103, 3.1607444, 3.2042508, 3.2042508, 2.159642 , 3.2789207,
       3.2789207, 3.39854  ], dtype=float32)

In [91]:
# setup grid search

param_grid = {
    'min_child_weight': [1, 3, 5, 10],
    'gamma': [.5, 1, 1.5, 2, 5],
    'subsample': [.4, .6, .8],
    'colsample_bytree': [.4, .6, .8],
    'max_depth': [3, 5, 7, 9]
}
print("There are", np.prod([len(x) for x in param_grid.values()]), "elements to search over")

There are  720  elements to search over


In [109]:
xgb_mod = XGBRegressor(learning_rate = .2, 
                        n_estimators = 600, 
                        objective = 'reg:squarederror', 
                        silent = True, 
                        nthread = 1)

In [117]:
Y = train['Y'].values
X = train.drop(columns = ['Y', 'system_key'])

# init small values for testing, even small numbers will take a long time here
nfolds = 3
param_comb = 4

kf = KFold(n_splits = nfolds, shuffle = True, random_state = 1001)
random_search = RandomizedSearchCV(xgb_mod, 
                                   param_distributions = param_grid, 
                                   n_iter = param_comb,
                                   scoring = 'neg_mean_absolute_error',
                                   n_jobs = 4,
                                   cv = kf.split(X, Y),
                                   verbose = 3,
                                   random_state = 1001)

random_search.fit(X, Y)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  12 | elapsed: 39.3min remaining:  7.9min
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 44.7min finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x1a263e0e60>,
                   error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.2, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=600,
                                          n_jobs=1...
                                          seed=None, silent=True, subsample=1,
                                          verbosity=1),
                   iid='warn', n_iter=4, n_jobs=4,
                   param_distributions={'colsample_bytree': [0.4, 0

In [130]:
# display results
print("CV results:\n", random_search.cv_results_)
print("Best estimator:\n", random_search.best_estimator_)
print("Best params:\n", random_search.best_params_)

CV results:
 {'mean_fit_time': array([ 697.63483334,  897.80643066, 1115.79682565,  526.9715356 ]), 'std_fit_time': array([ 1.02768782, 25.4935099 , 18.40182972, 25.14317307]), 'mean_score_time': array([0.91902868, 1.53031905, 1.39672788, 0.66214331]), 'std_score_time': array([0.03712481, 0.07450677, 0.18955454, 0.06344389]), 'param_subsample': masked_array(data=[0.4, 0.6, 0.6, 0.8],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[10, 10, 10, 10],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[9, 9, 9, 7],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[2, 0.5, 0.5, 5],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'param_colsample_bytree': masked_array(data=[0.4, 0.6, 0.

In [135]:
# pickle the result to avoid re-training
dump(random_search.best_estimator_, 'xgb-random-grid-search-test-best-estimator.joblib')

['xgb-random-grid-search-test-best-estimator.joblib']

In [125]:
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-test-results.csv', index = False)

In [142]:
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_min_child_weight,param_max_depth,param_gamma,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,697.634833,1.027688,0.919029,0.037125,0.4,10,9,2,0.4,"{'subsample': 0.4, 'min_child_weight': 10, 'ma...",-0.124457,-0.123343,-0.127873,-0.125224,0.001927,3
1,897.806431,25.493510,1.530319,0.074507,0.6,10,9,0.5,0.6,"{'subsample': 0.6, 'min_child_weight': 10, 'ma...",-0.113803,-0.112265,-0.111868,-0.112645,0.000834,2
2,1115.796826,18.401830,1.396728,0.189555,0.6,10,9,0.5,0.8,"{'subsample': 0.6, 'min_child_weight': 10, 'ma...",-0.110845,-0.110266,-0.109889,-0.110333,0.000393,1
3,526.971536,25.143173,0.662143,0.063444,0.8,10,7,5,0.4,"{'subsample': 0.8, 'min_child_weight': 10, 'ma...",-0.133128,-0.133969,-0.136625,-0.134574,0.001490,4


In [129]:
print(bst.predict(dtest)[:10])
print(random_search.predict(test.drop(columns = ['Y', 'system_key', 'Unnamed: 0'])[:10]))
print("truth:", test_y[:10])

[3.4236171 2.9237692 3.6941247 3.6941247 3.4306018 3.2245584 3.0839188
 3.6941247 2.926634  3.3301504]
[3.7365222 3.2690427 3.6912916 3.7557857 3.6505923 3.6093075 3.007139
 3.5424728 2.891696  3.5847788]
truth: 0     3.570652
1     3.500000
4     3.748780
5     3.811111
6     3.726316
7     3.709524
8     3.021429
9     3.315385
10    2.901587
11    3.680851
Name: Y, dtype: float64


#### Brute force solution

In [ ]:
# grid = GridSearchCV(estimator=xgb, 
param_grid=params, scoring='roc_auc', n_jobs=4, cv=skf.split(X,Y), verbose=3 )
# grid.fit(X, Y)
# print('\n All results:')
# print(grid.cv_results_)
# print('\n Best estimator:')
# print(grid.best_estimator_)
# print('\n Best score:')
# print(grid.best_score_ * 2 - 1)
# print('\n Best parameters:')
# print(grid.best_params_)
# results = pd.DataFrame(grid.cv_results_)
# results.to_csv('xgb-grid-search-results-01.csv', index=False)

# y_test = grid.best_estimator_.predict_proba(test)
# results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
# results_df.to_csv('submission-grid-search-xgb-porto-01.csv', index=False)